In [1]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
import torch
import torch
model_id = 'naver/splade-cocondenser-ensembledistil'

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForMaskedLM.from_pretrained(model_id)

In [2]:
tokens = tokenizer("query", return_tensors='pt')
output = model(**tokens)
output

MaskedLMOutput(loss=None, logits=tensor([[[ -7.0360,  -8.6398,  -7.8778,  ...,  -7.9991,  -7.9456,  -7.5978],
         [-29.9240, -20.6621, -20.0392,  ..., -20.2664, -20.2275, -27.9713],
         [-17.2971, -14.5340, -13.8390,  ..., -14.4820, -13.3104, -15.0416]]],
       grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)

In [32]:
def sim_test(query):
    tokens = tokenizer(query, return_tensors='pt')
    output = model(**tokens)
    # output.logits.shape
    vec = torch.max(
        torch.log(
            1 + torch.relu(output.logits)
        ) * tokens.attention_mask.unsqueeze(-1),
    dim=1)[0].squeeze()
    # extract non-zero positions
    cols = vec.nonzero().squeeze().cpu().tolist()
    print(len(cols))
    # extract the non-zero values
    weights = vec[cols].cpu().tolist()
    # use to create a dictionary of token ID to weight
    sparse_dict = dict(zip(cols, weights))
    idx2token = {
        idx: token for token, idx in tokenizer.get_vocab().items()
    }
    sparse_dict_tokens = {
        idx2token[idx]: round(weight, 2) for idx, weight in zip(cols, weights)
    }

    # sort so we can see most relevant tokens first
    sparse_dict_tokens = {
        k: v for k, v in sorted(
            sparse_dict_tokens.items(),
            key=lambda item: item[1],
            reverse=True
        )
    }
    return sparse_dict_tokens
    # vec.shape
sim_test("war")

48


{'war': 2.97,
 'wars': 1.92,
 'conflict': 1.01,
 'battle': 0.96,
 'army': 0.71,
 'peace': 0.67,
 'weapon': 0.51,
 'owen': 0.48,
 'iraq': 0.44,
 'warfare': 0.44,
 'slavery': 0.41,
 'threat': 0.38,
 'anger': 0.32,
 'event': 0.3,
 'fight': 0.3,
 'victor': 0.29,
 'preston': 0.29,
 'soldier': 0.28,
 'football': 0.27,
 'political': 0.27,
 'marshall': 0.25,
 'wilson': 0.24,
 'game': 0.23,
 'alliance': 0.22,
 'treaty': 0.21,
 'bradley': 0.2,
 'savage': 0.19,
 'military': 0.18,
 'benjamin': 0.17,
 'germany': 0.16,
 'nelson': 0.13,
 'storm': 0.11,
 'race': 0.1,
 'trade': 0.09,
 'zombie': 0.08,
 'aftermath': 0.07,
 'strategy': 0.06,
 'hammer': 0.05,
 'ruin': 0.05,
 'bomb': 0.04,
 'clash': 0.03,
 'enemy': 0.02,
 'combat': 0.02,
 'story': 0.01,
 'confederate': 0.01,
 'gun': 0.0,
 'foster': 0.0,
 'ally': 0.0}

In [33]:
# vec

In [33]:
import torch
from transformers import BertModel, BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-large-cased-whole-word-masking')
model = BertModel.from_pretrained('bert-large-cased-whole-word-masking')

In [4]:
from transformers import BertTokenizer, BertForMaskedLM
import torch

# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

print(len(tokenizer))  # 28996
# tokenizer.add_tokens(["NEW_TOKEN"])
# print(len(tokenizer))  # 28997

# model.resize_token_embeddings(len(tokenizer)) 


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


28996


In [3]:
# Prepare input
input_text = "The capital of France is [MASK]."
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# Predict masked token
with torch.no_grad():
    outputs = model(input_ids)
    predictions = outputs.logits

# Get predicted token
predicted_index = torch.argmax(predictions[0, input_ids[0] == tokenizer.mask_token_id], dim=1)
predicted_token = tokenizer.decode(predicted_index)
predicted_token 

NameError: name 'torch' is not defined

In [ ]:
len(output.logits[0])

3

In [30]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))

def process(text):
    # Remove special characters
    text = re.sub(r'\W', ' ', text)
    
    word_tokens = word_tokenize(text)
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
    
    return " ".join(filtered_sentence)


In [36]:
from langchain.document_loaders import DirectoryLoader, TextLoader
text = DirectoryLoader('/Users/praveenlawyantra/Desktop/icc_all/all_refined', glob="./*_refined.txt", loader_cls=TextLoader)
documents = text.load()
un = DirectoryLoader('/Users/praveenlawyantra/Desktop/un_all/all_refined', glob="./*_refined.txt", loader_cls=TextLoader)
un_d= un.load()

ass=[]
for i in documents:
    ass.append(i)
for i in un_d:
    ass.append(i)

print(len(ass))

preocessed_data=[process(i.page_content) for i in ass]
# preocessed_data
def tokenize_text(text, tokenizer):
    tokens = tokenizer.tokenize(text)
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    # output=model(**tokens)
    return tokens, token_ids

# Initialize an empty dictionary to store the combined vocabulary
combined_vocab = {}
vocabb=[]
# Assuming `processed_data` is a list of texts
for text in preocessed_data:
    tokens, token_ids = tokenize_text(text, tokenizer)
    # print(output)
    vocabb.append(tokens)
    # Update the combined vocabulary with tokens and their IDs from the current text
    combined_vocab.update(zip(tokens, token_ids))

print("Combined Vocabulary:", combined_vocab)
len(combined_vocab)

In [23]:
base_file = open('combined_output.txt', 'rt')
raw_text = base_file.read()
# raw_text = nltk.corpus.gutenberg.raw('/Users/praveenlawyantra/Desktop/Search-Engine-API/train.py')
base_file.close()
def tokenize_text(text, tokenizer):
    tokens = tokenizer.tokenize(text)
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    # output=model(**tokens)
    # print(tokens,token_ids)
    return str(tokens), token_ids

In [24]:
tokens, token_ids = tokenize_text(raw_text, tokenizer)
# print(output)


In [29]:
print(len(tokens))
# combined_vocab = {}
vocab=[]

53781125


In [30]:
print(len(tokens))
print(len(token_ids))



53781125
7489215


In [31]:
for i in tokens:
    vocab.append(str(i))

In [32]:
print(len(vocab))

53781125


In [68]:
for i,j in zip(tokens, token_ids):
    # print(i,j)
    combined_vocab.update(zip([i], [j]))

In [70]:

print("Combined Vocabulary:", combined_vocab)
len(combined_vocab)

Combined Vocabulary: {'case': 2553, 'details': 4751, ':': 1024, '-': 1011, 'id': 8909, 'icc': 16461, '##01': 24096, '/': 1013, '14': 2403, '01': 5890, '22': 2570, 'court': 2457, 'tribunal': 12152, 'international': 2248, 'criminal': 4735, 'document': 6254, 'date': 3058, '17': 2459, 'october': 2255, '202': 16798, '##3': 2509, 'key': 3145, 'parties': 4243, 'involved': 2920, 'the': 1996, 'prosecutor': 12478, 'mr': 2720, '.': 1012, 'maxim': 20446, '##e': 2063, 'je': 15333, '##off': 7245, '##roy': 13238, 'eli': 12005, 'mo': 9587, '##ko': 3683, '##m': 2213, 'ga': 11721, '##wa': 4213, '##ka': 2912, '(': 1006, 'defendant': 13474, ')': 1007, 'name': 2171, 'v': 1058, 'representatives': 4505, 'of': 1997, 'victim': 6778, 'abd': 19935, '##ou': 7140, 'dan': 4907, '##ga': 3654, '##bo': 5092, '##uss': 17854, '##a': 2050, 'ms': 5796, 'marie': 5032, 'edith': 13257, 'do': 2079, '##uz': 17040, '##ima': 9581, 'lawson': 14577, 'ya': 8038, '##r': 2099, 'fall': 2991, 'applicant': 23761, 'karim': 25461, 'a': 10

13053

In [33]:
# vocabb
t=list(set(tokens))
len(t)

119

In [37]:
print(len(tokenizer))  # 28996
print(len(tokens))  # 28996

# tokenizer.add_tokens("okyee")
# tokens, token_ids = tokenize_text(raw_text, tokenizer)
# print(output)

for i in tokens:
#     # print(i)
#     w=str(i)
    tokenizer.add_tokens(str(i))

    print(len(tokenizer))  # 28997
    model.resize_token_embeddings(len(tokenizer)) 


28997
53781125
28997
28997
28997
28997
28997
28997


KeyboardInterrupt: 

In [28]:
tokens = tokenizer("query", return_tensors='pt')
output = model(**tokens)
output

MaskedLMOutput(loss=None, logits=tensor([[[ -7.5507,  -7.4356,  -7.4750,  ...,  -0.0567,  -0.2864,   0.2440],
         [ -9.4717,  -9.2587,  -9.6860,  ...,   0.0857,  -0.5279,   1.3659],
         [ -9.9250,  -9.7454, -10.2649,  ...,   0.1101,   0.5838,   1.4846],
         [-11.1762, -11.0812, -11.0485,  ...,  -0.9995,  -0.0334,   3.3140]]],
       grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)

In [29]:
def sim_test(query):
    tokens = tokenizer(query, return_tensors='pt')
    output = model(**tokens)
    # output.logits.shape
    vec = torch.max(
        torch.log(
            1 + torch.relu(output.logits)
        ) * tokens.attention_mask.unsqueeze(-1),
    dim=1)[0].squeeze()
    # extract non-zero positions
    cols = vec.nonzero().squeeze().cpu().tolist()
    print(len(cols))
    # extract the non-zero values
    weights = vec[cols].cpu().tolist()
    # use to create a dictionary of token ID to weight
    sparse_dict = dict(zip(cols, weights))
    idx2token = {
        idx: token for token, idx in tokenizer.get_vocab().items()
    }
    sparse_dict_tokens = {
        idx2token[idx]: round(weight, 2) for idx, weight in zip(cols, weights)
    }

    # sort so we can see most relevant tokens first
    sparse_dict_tokens = {
        k: v for k, v in sorted(
            sparse_dict_tokens.items(),
            key=lambda item: item[1],
            reverse=True
        )
    }
    return sparse_dict_tokens
    # vec.shape
sim_test("war")

856


{'.': 2.84,
 ';': 2.61,
 ',': 2.49,
 'war': 2.45,
 '?': 2.27,
 ')': 2.08,
 "'": 2.02,
 'the': 1.97,
 '!': 1.96,
 '|': 1.86,
 '-': 1.84,
 'peace': 1.81,
 'more': 1.74,
 '...': 1.71,
 '"': 1.68,
 'conflict': 1.64,
 'of': 1.62,
 'wars': 1.59,
 '[\'summary\', \':\', \'case\', \'details\', \':\', \'-\', \'case\', \'id\', \':\', \'icc\', \'-\', \'02\', \'/\', \'05\', \'-\', \'01\', \'/\', \'20\', \'-\', \'court\', \'/\', \'tribunal\', \':\', \'international\', \'criminal\', \'court\', \'-\', \'document\', \'date\', \':\', \'5\', \'december\', \'202\', \'##2\', \'-\', \'key\', \'parties\', \'involved\', \':\', \'-\', \'the\', \'prosecutor\', \'-\', \'ali\', \'muhammad\', \'ali\', \'abd\', \'##al\', \'-\', \'rahman\', \'(\', \'`\', \'`\', \'ali\', \'ku\', \'##sha\', \'##y\', \'##b\', "\'", "\'", \')\', \'representatives\', \'victim\', \':\', \'not\', \'mentioned\', \'representatives\', \'applicant\', \':\', \'not\', \'mentioned\', \'procedural\', \'history\', \':\', \'-\', \'the\', \'case\', \

In [13]:

# from collections import defaultdict
# sparse_values = defaultdict(set)

# Assuming `processed_data` is a list of texts
for text in preocessed_data:
    tokens, token_ids = tokenize_text(text, tokenizer)
    # Update the combined vocabulary with tokens and their IDs from the current text
    combined_vocab.update(zip(tokens, token_ids))
    # Update sparse values
    # for token, token_id in zip(tokens, token_ids):
    #     sparse_values[token_id].add(token)

print("Combined Vocabulary:", combined_vocab)
# print("Sparse Values:", sparse_values)
print("Length of Combined Vocabulary:", len(combined_vocab))

# for i,j in combined_vocab.items():
#     # vocabb.append(i)
#     print(i,":",j)

Combined Vocabulary: {'case': 2553, 'details': 4751, ':': 1024, '-': 1011, 'id': 8909, 'icc': 16461, '##01': 24096, '/': 1013, '14': 2403, '01': 5890, '21': 2538, 'court': 2457, 'tribunal': 12152, 'international': 2248, 'criminal': 4735, '(': 1006, ')': 1007, 'document': 6254, 'date': 3058, '8': 1022, 'december': 2285, '202': 16798, '##2': 2475, 'key': 3145, 'parties': 4243, 'involved': 2920, 'the': 1996, 'prosecution': 11537, ',': 1010, 'defence': 4721, 'name': 2171, 'prosecutor': 12478, 'v': 1058, '.': 1012, 'maha': 24404, '##mat': 18900, 'said': 2056, 'abd': 19935, '##el': 2884, 'kan': 22827, '##i': 2072, 'representatives': 4505, 'victim': 6778, 'applicant': 23761, 'ms': 5796, 'jennifer': 7673, 'na': 6583, '##our': 8162, 'mr': 2720, 'do': 2079, '##v': 2615, 'jacobs': 12988, 'legal': 3423, 'representative': 4387, 'victims': 5694, 'ma': 5003, '##me': 4168, 'man': 2158, '##dia': 9032, '##ye': 6672, 'ni': 9152, '##ang': 5654, 'counsel': 9517, 'procedural': 24508, 'history': 2381, 'previ

In [38]:
from gensim.models import Word2Vec
import gensim
model1 = gensim.models.Word2Vec(vocabb, min_count=1,vector_size=1000, window=50)
model1.build_vocab(vocabb, progress_per=1000)
model1.train(vocabb, total_examples=model1.corpus_count, epochs=model1.epochs)


In [ ]:
model1.save("word2vec.model")


In [ ]:
import numpy as np
vectors = np.array([model1.wv[word] for word in model1.wv.key_to_index])

def cosine_similarity(vec_a, vec_b):
    """Compute cosine similarity between two vectors."""
    cos_sim = np.dot(vec_a, vec_b) / (np.linalg.norm(vec_a) * np.linalg.norm(vec_b))
    return cos_sim

# Example vectors
def get_similirity(query):
    word_vector = np.array(model1.wv[query])  # Vector for the selected word
    other_vectors = np.array(vectors)  # Vectors for other words in your corpus

    # Compute similarity weights
    similarity_weights = [cosine_similarity(word_vector, vec) for vec in other_vectors]
    sparse_dict_tokens = dict(zip(model1.wv.key_to_index, similarity_weights))
    # for word, sim in zip(model1.wv.key_to_index, similarity_weights):
    #     print(f"Word: {word}, Similarity: {sim}")
    sparse_dict_tokens

    sparse_dict_tokens = {
        k: v for k, v in sorted(
            sparse_dict_tokens.items(),
            key=lambda item: item[1],
            reverse=True
        )
    }
    return sparse_dict_tokens
get_similirity("war")

{'war': 0.9999999,
 'committed': 0.6656528,
 'crime': 0.42708167,
 'Crime': 0.41014513,
 'humanity': 0.39975926,
 'scheduling': 0.38885334,
 'approval': 0.37804037,
 'prosecution': 0.36355782,
 'applicable': 0.36157432,
 'Flores': 0.35687718,
 'first': 0.3561033,
 'Fiona': 0.35397312,
 'charged': 0.34537423,
 'Place': 0.3131442,
 'pm': 0.31129742,
 '##ra': 0.3100567,
 'counts': 0.30550396,
 'authorization': 0.3029427,
 'internal': 0.2977947,
 '##tien': 0.28926677,
 'leader': 0.28885084,
 '##makers': 0.28830862,
 'northern': 0.287192,
 '1872': 0.28615016,
 'weapon': 0.2849641,
 'impact': 0.28344408,
 'procession': 0.28222,
 'objects': 0.28133273,
 '##lant': 0.28042334,
 'rescue': 0.27947012,
 'Michel': 0.2769246,
 'Chung': 0.2731184,
 'alter': 0.27310616,
 'Post': 0.27264544,
 '##NE': 0.27205193,
 'lifetime': 0.27177024,
 'managing': 0.26809564,
 'charges': 0.2676657,
 'responsible': 0.26654953,
 'trouble': 0.26491982,
 'b': 0.2644447,
 'x': 0.262345,
 'Fifth': 0.2615874,
 'hostilities'

In [1]:
!pip install PyMuPDF


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 25.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 18.6 MB/s eta 0:00:0000:0100:01


In [15]:
import fitz  # PyMuPDF
import requests


def download_pdf(url, filename):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(response.content)
        print("Download successful.")
        return True
    else:
        print("Download failed with status code:", response.status_code)
        return False

def highlight_words_in_pdf(pdf_path, output_path, word, color=[1, 0, 0]):
    try:
        doc = fitz.open(pdf_path)
        for page in doc:
            text_instances = page.search_for(word)
            for inst in text_instances:
                highlight = page.add_highlight_annot(inst)
                highlight.set_colors(stroke=color)
                highlight.update()
        doc.save(output_path)
        return True
    except Exception as e:
        print(f"Failed to highlight words in the PDF: {e}")
        return False

def main(pdf_url, local_pdf_path, highlighted_pdf_path, word_to_highlight):
    if download_pdf(pdf_url, local_pdf_path):
        if highlight_words_in_pdf(local_pdf_path, highlighted_pdf_path, word_to_highlight):
            print(f"Highlighted PDF saved as: {highlighted_pdf_path}")
        else:
            print("Failed to highlight words in the PDF.")
    else:
        print("Failed to download the PDF.")

# The provided PDF URL
pdf_url = 'https://www.icc-cpi.int/sites/default/files/CourtRecords/CR2021_05611.PDF'
local_pdf_path = 'downloaded_document.pdf'  # Temp file for downloaded PDF

highlighted_pdf_path = 'downloaded_document.pdf'  # Final output file
word_to_highlight = 'evidence'  # Example word to highlight

main(pdf_url, local_pdf_path, highlighted_pdf_path, word_to_highlight)


Download successful.
Failed to highlight words in the PDF: save to original must be incremental
Failed to highlight words in the PDF.


In [18]:

pdf_url = 'https://www.icc-cpi.int/sites/default/files/CourtRecords/CR2021_05611.PDF'
print(pdf_url.split("/")[-1:])

['CR2021_05611.PDF']


In [ ]:
from collections import defaultdict
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from transformers import BertTokenizer, BertModel
import torch
from rank_bm25 import BM25Okapi

# Initialize tokenizer and model once
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Assuming stop_words and punctuation are defined
stop_words = set(stopwords.words('english'))

def process(text):
    word_tokens = word_tokenize(text)
    return " ".join(w for w in word_tokens if w.lower() not in stop_words)

def tokenize_documents(documents):
    tokenized_docs = [tokenizer.tokenize(doc) for doc in documents]
    return BM25Okapi(tokenized_docs)

def predict_term_weights(query):
    inputs = tokenizer(query, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    weights = torch.mean(outputs.last_hidden_state, dim=1).squeeze().numpy()
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"].squeeze().tolist())
    return dict(zip(tokens, weights))

def search(docs, query):
    bm25 = tokenize_documents(docs)
    term_weights = predict_term_weights(query)
    scores = defaultdict(float)
    for term, weight in term_weights.items():
        term_scores = bm25.get_scores([term])
        for doc_id, score in enumerate(term_scores):
            scores[doc_id] += score * weight
    return scores, sorted(scores, key=scores.get, reverse=True)

def searchh(texttt):
    # Placeholder for retriever.get_relevant_documents method
    pine = []  # Replace with actual call
    content = [i.page_content for i in pine]
    procesedd_text = [process(content[i]) for i in range(len(content))]
    return pine, procesedd_text

def results(query):
    pine, texts = searchh(query)
    scores, ids = search(texts, query)
    resultss = [(round(float(scores[i])*100 + 100, 2), pine[i]) for i in ids]
    return resultss, ids

# Example usage:
# resultss, ids = results("Example query")
# print(resultss)


In [19]:
def generate_ngrams(text, n):
    """
    Generate n-grams from the given text.
    
    :param text: The input text string.
    :param n: The number of words in each n-gram.
    :return: A list of n-grams.
    """
    # Ensure the text is a list of words
    words = text.split()
    
    # Generate n-grams
    ngrams = [words[i:i+n] for i in range(len(words)-n+1)]
    
    return ngrams

# Example usage
text = "This is a simple example of generating n-grams from text."
n = 2  # For bigrams
ngrams = generate_ngrams(text, n)
print(ngrams)

n = 3  # For trigrams
ngrams = generate_ngrams(text, n)
print(ngrams)


[['This', 'is'], ['is', 'a'], ['a', 'simple'], ['simple', 'example'], ['example', 'of'], ['of', 'generating'], ['generating', 'n-grams'], ['n-grams', 'from'], ['from', 'text.']]
[['This', 'is', 'a'], ['is', 'a', 'simple'], ['a', 'simple', 'example'], ['simple', 'example', 'of'], ['example', 'of', 'generating'], ['of', 'generating', 'n-grams'], ['generating', 'n-grams', 'from'], ['n-grams', 'from', 'text.']]


In [26]:
import pickle
import requests
 
from flask import  jsonify

In [24]:
with open('ngram_model1.pickle', 'rb') as handle:
    model = pickle.load(handle)



[nltk_data] Downloading package punkt to
[nltk_data]     /Users/praveenlawyantra/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/praveenlawyantra/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [32]:
model.suggest("legal issues")

{'next_words': [{'token': 'addressed alleged war crimes and crimes against humanity.',
   'score': 1},
  {'token': 'addressed alleged war crimes and crimes against humanity in the central african republic.',
   'score': 1},
  {'token': 'addressed', 'score': 2.0}],
 'replacements': []}

In [28]:
def suggest():
    # content = request.json()
    phrase = "legal"
    return jsonify(model.suggest(phrase))
suggest()

RuntimeError: Working outside of application context.

This typically means that you attempted to use functionality that needed
the current application. To solve this, set up an application context
with app.app_context(). See the documentation for more information.

In [29]:
from collections import defaultdict

def build_conditional_probabilities(corpus):
	"""
	The function takes as its input a corpus string (words separated by 
	spaces) and returns a 2D dictionnary of probabilities P(next|current) of
	seeing a word "next" conditionnaly to seeing a word "current". 
	"""

	# First we parse the string to build a double dimension dictionnary that
	# returns the conditional probabilities.

	# We parse the string to build a first dictionnary indicating for each
	# word, what are the words that follow it in the string. Repeated next
	# words are kept so we use a list and not a set. 

	tokenized_string = corpus.split()
	previous_word = ""
	dictionnary = defaultdict(list)

	for current_word in tokenized_string:
		if previous_word != "":
			dictionnary[previous_word].append(current_word)
		previous_word = current_word
		
	# We know parse dictionnary to compute the probability each observed
	# next word for each word in the dictionnary. 

	for key in dictionnary.keys():
		next_words = dictionnary[key]
		unique_words = set(next_words) # removes duplicated
		nb_words = len(next_words)
		probabilities_given_key = {}
		for unique_word in unique_words:
			probabilities_given_key[unique_word] = \
				float(next_words.count(unique_word)) / nb_words
		dictionnary[key] = probabilities_given_key

	return dictionnary


def bigram_next_word_predictor(conditional_probabilities, current, next_candidate):
	"""
	The function takes as its input a 2D dictionnary of probabilities 
	P(next|current) of seeing a word "next" conditionnaly to seeing a word 
	"current", the current word being read, and a next candidate word, and
	returns P(next_candidate|current).
	"""

	# We look for the probability corresponding to the 
	# current -> next_candidate pair

	if conditional_probabilities.has_key(current):
		if conditional_probabilities[current].has_key(next_candidate):
			return conditional_probabilities[current][next_candidate]

	# If current -> next_candidate pair has not been observed in the corpus,
	# the corresponding dictionnary keys will not be defined. We return 
	# a probability 0.0

	return 0.0

# An example corpus to try out the function
corpus = "the cat is red the cat is green the cat is blue the dog is brown"

# We call the conditional probability dictionnary builder function
conditional_probabilities = build_conditional_probabilities(corpus)

# Some sample queries to the bigram predictor
assert bigram_next_word_predictor(conditional_probabilities, "the", "cat") == 0.75
assert bigram_next_word_predictor(conditional_probabilities, "is", "red") == 0.25
assert bigram_next_word_predictor(conditional_probabilities, "", "red") == 0.0


AttributeError: 'collections.defaultdict' object has no attribute 'has_key'